# Lesson 9: Class-Based Views
---
Intro: We will look at various class-based views in Django and apply them to our website.

# Review
---

1. In your own words what is a decorator?
2. What does the render function do?
3. In general, what are the differences between a class and a function?

# Concept 1: Class Based Views vs Function Based Views
---


## What are they?

We have been working with function based views this whole time. Here's what a function based view looks like:

```
def about(request):
    return render(request, 'blog/about.html', {'title': 'About'})
```
This is a function based view because it is a function and it renders a view - the about.html file.

Now let's take a look at a class based view:
```
class PostListView(ListView)
    model = Post
```
* We create a class called PostListView that inherits from ListView. 
* The model interacts with the view so it can decide what to render as a view.

Just by inheriting from ListView, it renders the page and handles the HTTP requests. Essentially it does all the background work for us. Django recognizes that many websites in production need to display lists so they made it easier to create a list view or any view of some sort. Django calls these "generic views" because they provide solutions to common requirements. So why ListView? We are making a blog website so we need to display our list of posts.

There are different Views that we can inherit such as:
* DetailView
* CreateView
* DeleteView
* TemplateView
* FormView
* ListView

In its pure form, they are Python classes but once they need to be rendered then we need to cast them as views using `as_view()`.

So when do you use class based views and function based views? 
> It depends! 
* If you want something easier to read or require decorators, then function based views are better. However, it's a pain to reuse this code and HTTP requests must be handled manually.
* If you want to use Django's views or reuse code, then class based views are better. However, it's harder to read if you don't truly understand what's going on.

## Examples:
---

In [ ]:
# Function based view
def about(request):
    return render(request, 'blog/about.html', {'title': 'About'})

In [ ]:
# Class based view
class PostListView(ListView)
    model = Post

## DIY:
---

1. What are the components of a function based view?
2. What are the components of a class based view?
3. In a class based view, what do you inherit?
4. What are the differences between CBV and FBV?

# Concept 2: ListView
---


## Objective
Let's create our class based view. As we talked about earlier, Django has a ListView so we can use that to display our list of posts. We are going to transform the home function to become a class based view.

## PostListView
1. `from django.views.generic import ListView` - At the top, let's import the list view.
2. `class PostListView(ListView):` - Create a class that inherits from ListView 
3. `model = Post` - Create a variable called model. This will tell our list view what model to examine to create the list. We set this to all of our posts. Technically, this is all we need to create a class based view but we will add more details in a bit.
4. Open the `blog/urls.py` file. Instead of using `views.home`, we will use the PostListView. 
5. `from .views import PostListView` - So first we need to import it. 
6. Now replace `views.home` with `PostListView.as_view()`.
> Answer this: why do we need the as_view function here?
7. Let's run the server at `localhost:8000`.
8. There is an error! By default, class-based views look for templates of a certain naming pattern. Take a look at the error again. This naming pattern is this: `<app>/<model>_<viewtype>.html`
9. `template_name = 'blog/home.html'` - Since we have a template already, we just provide its name.
10. `context_object_name = 'posts'` - Remember we also have a context dictionary. Django needs to loop over our posts. We simply provide a context object name and assign it to our list of posts.
11. `ordering = ['-date_posted']` - Now since we have a list of blog posts, we want our most recent posts at the top. This orders the posts by the date posted and the negative sign just means to go from newest to oldest.
12. Let's run it in the browser! Try make another post in the admin page to see if the ordering works.

> Take a look at the home function-based view and PostListView class-based view. In home(), we had to render everything out and provide a context dictionary. In PostListView, we simply assign variables.


## DIY:
---

1. For our list of blog posts, what type of class-based view are we using?
2. Why do we need `as_view()`?

# Concept 3: DetailView
---


## Objective
Let's now create a view for individual posts. This is going to be a [DetailView](https://docs.djangoproject.com/en/3.1/ref/class-based-views/generic-display/#generic-display-views) because we would like to see details about the post.
## PostDetailView
13. On the same line where you imported ListView,  include `DetailView`
14. `class PostDetailView(DetailView):` - Create a class called PostDetailView that inherits from DetailView.
15. `model = Post` - This detail view will use each post and reveal its details.
16. Open `blog/urls.py`. We need to create a route that takes us to a specific post. We haven't seen this yet so we have to create a URL pattern that contains a variable. Example: `localhost:8000/post/1/`, `localhost:8000/post/2/`, etc.
17. First off, along with PostListView, import `PostDetailView`
18. Now to create the variable in our URL path we will use an auto-incrementing primary key. Add this to the urlpatterns list:
```
path('post/<int:pk>/', PostDetailView.as_view(), name='post-detail'),
```
* `<int:pk>` - This is the primary key stated as an integer. The primary key will always be an integer and not any other data type.
19. Now create a template that displays our post details. Recall that our generic class-based view is looking for a template with this naming convention `<app>/<model>_<viewtype>.html`. For our example, it needs to look for `blog/post_detail.html`.
20. In `blog/templates/blog`, create a template called `post_detail.html`. A single post's details are revealed here.
21. Let's copy `home.html` and paste it into `post_detail.html`. This already has the styling for multiple posts so now we just need it for one post.
22. Remove the for loop and the end for.
23. Take a look at the h2 tag containing `{{ post.title}}`. We don't need the a tag anymore because we don't need to link anything to the title. Change it to this - `<h2 class="article-title">{{post.title}}</h2>`
24. One more change. When dealing with detail views, it expects the context of this template to be called `object`. So change all post variables to use `object`. There are 5 post variables to replace.
25. Let's open the browser and check out `localhost:8000/post/1/`. What do you see?
26. Go to `localhost:8000` and try click on the blog titles. Nothing happens but we will fix that right now.
27. Open the `home.html` template and go to the line containing `{{post.title}}`. Edit the href so it can link to each post using `href="{% url 'post-detail' post.id %}"`
* Post.id is the primary key we saw earlier when setting up the url paths
28. Reload `localhost:8000` and click on the post titles. 

## DIY:
---

1. What is a primary key?
2. In your own words, describe DetailView.

# Concept 4: CreateView
---


## Objective
Before, only the admin can create posts by going on the admin page. Let's create a view where users can create posts in the blog itself.
## CreateView
29. First off, go [here](https://docs.djangoproject.com/en/3.1/ref/class-based-views/generic-editing/) to read more about the generic views.
30. Open `views.py`, and include `CreateView` with all the rest of the views.
31. `class PostCreateView(CreateView):` - Create a class called PostCreateView that inherits from CreateView. So when a user wants to create a post, they need to enter a title and the content. CreateView handles the model and the fields to create the view selected.
32. `model = Post` - The Post model handles what goes on behind the scenes
33. `fields = ['title','content']` - These input fields are necessary when creating a post.
34. Since we're finished with that part, go to the urls.py page and create a url pattern for this - `path('post/new/', PostCreateView.as_view(), name='post-create')`
35. We now need a template to go along with this. The UpdateView that we will create later will share this template so let's name it `post_form.html` and put it in `blog/templates/blog`.
36. As a starting point, copy the register page from users/ and paste it onto post_form.html.
37. Change the legend to say Blog Post instead of Join Today. Change the button to say Post instead of Sign up. Delete the entire div containing the class border-top pt-3. With all these changes, we just have the form available.
38. Now open the webiste on the browser and head to `localhost:8000/post/new/`. We can't create a post yet because we didn't set up the button nor the validation checks.
39. So head over to `views.py` and within the PostCreateView, create this function
```
def form_valid(self, form):
      form.instance.author = self.request.user
      return super().form_valid(form)
```
* `form.instance.author = self.request.user` - Set the current author logged in to be the author of the post that will be created
* `return super().form_valid(form)` - This uses the base class (CreateView's) form_valid method that simply checks if a form is valid given the current author
40. Let's run the browser again and try create a new post at `localhost:8000/post/new/`. What happens?
41. When you click submit. We get an error saying we must provide a url or define a get_absolute_url method on the Model. We will do just that.

> Difference between relative url and absolute url:
An absolute URL contains all the information necessary to locate a resource.
Ex: https://www.google.com/images/id/383/cats
A relative URL provides only the tag of an absolute URL. 
Ex: /383/cats

Click this [picture](https://images.app.goo.gl/azNRNV5xPZeDjtLTA)

42. Open the blog's `models.py` and create this method:

```
from django.urls import reverse

def get_absolute_url(self):
    return reverse('post-detail', kwargs={'pk':self.pk})
```
* Reverse will return the full path as a string. The path that we want to get is the path to the post-detail route. We need a specific post with a primary key so we set our [kwargs](https://www.geeksforgeeks.org/args-kwargs-python/) value or keyword argument equal to the primary key.


43. Let's go to the browser again and create a new post. This should work now!

## Mixins

44. One more thing, we need to make sure that we are logged in to create a post. Otherwise, we will just be redirected to the login page. Since we are using class-based views we can't use decorators anymore because those are reserved for functions. We will use mixins. Mixins allow for multiple inheritances for a single class. Put it simply, mixins are basically decorators for classes. Put it even more simply, mixins provide more features.
45. Go to the views.py page and include this:
`from django.contrib.auth.mixins import LoginRequiredMixin`
46. So Python allows for multiple inheritance so in the PostCreateView on the far left of what's being passed in put `LoginRequiredMixin`. It becomes this - `class PostCreateView(LoginRequiredMixin, CreateView)`
47. Now log out, and go to `localhost:8000/post/new/`. You should be redirected to the log in page.

## DIY:
---

1. Explain CreateView. What is necessary to have when using CreateView?
2. What are mixins?

# Summary:
---


1. What are class-based views?
2. What are function-based views?
3. Explain ListView
3. Explain DetailView
3. Explain CreateView
3. Explain UpdateView
3. Explain DeleteView

# Homework:
---



1. Continue working on the BlackJack game.

# Notes on homework:
---

I will check in on Thursday,  through email to check on your progress. Respond with any questions you might have. Otherwise, a simple “all good” is appropriate if you have no questions or comments. 

You will need to upload your coding homework assignments to GitHub.
1. In gitbash, change directories to the homework directory: tomas_python/homework
* TIP: use ‘cd’ to change directories
* Use ‘cd ..’ to return to the previous directory
* Use ‘pwd’ to show full pathname of the current working directory 
* Use ‘ls’ to list all your directories
2. Once you’re in that directory, type in ‘git pull’
* This ensures you have all updated files
* If there is an error involved, email me immediately so we can try resolving it.
* Otherwise, type your code below and we’ll resolve issues next class
3. To create a new file, type in ‘touch hw01.py’ or the appropriate file name
* ‘Touch’ creates a new file
4. Open up the python file and start coding!

Note: Become familiar with these actions. This is essentially what happens in the backend when you right-click and create a new folder/file!
